In [7]:
# GAN 모델을 이용해 단순히 랜덤한 숫자를 생성하는 아닌,
# 원하는 손글씨 숫자를 생성하는 모델을 만들어봅니다.
# 이런 방식으로 흑백 사진을 컬러로 만든다든가, 또는 선화를 채색한다든가 하는 응용이 가능합니다.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import OneHotEncoder

In [2]:
#########
# 옵션 설정
######
total_epoch = 13
batch_size = 1000
n_hidden = 15
n_input = 26
n_noise = 50
n_class = 2

In [3]:
# Original DataSet
#-- (20631,26)

ori_train = pd.read_csv('./data/train_FD0001_true5.csv',encoding= 'cp949')
ori_test = pd.read_csv('./data/test_FD0001_true5.csv',encoding= 'cp949')
data = pd.concat([ori_train,ori_test], axis = 0)
data_X = data.iloc[:,:-1].values.tolist()
data_y = data.iloc[:,-1]

data_y = data_y.replace('N',0)
data_y = data_y.replace('Y',1)
y_len = len(data_y)
data_y = tf.one_hot(data_y,depth = 2,dtype=tf.float32)

"""
https://hiseon.me/2018/04/15/tensorflow-dataset/
"""
dataset = tf.data.Dataset.from_tensor_slices((data_X, data_y))
dataset = dataset.shuffle(1000).repeat().batch(batch_size)


In [4]:
#########
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈와 실제 이미지에, 그에 해당하는 숫자에 대한 정보를 넣어주기 위해 사용합니다.
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [5]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 추가합니다.
        inputs = tf.concat([noise, labels], 1)

        # TensorFlow 에서 제공하는 유틸리티 함수를 이용해 신경망을 매우 간단하게 구성할 수 있습니다.
        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,
                                 activation=tf.nn.sigmoid)

    return output


def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기 위해,
        # 이전에 사용되었던 변수를 재사용하도록 합니다.
        if reuse:
            scope.reuse_variables()

        inputs = tf.concat([inputs, labels], 1)

        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1,
                                 activation=None)

    return output


def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

# 생성 모델과 판별 모델에 Y 즉, labels 정보를 추가하여
# labels 정보에 해당하는 이미지를 생성할 수 있도록 유도합니다.
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# 손실함수는 다음을 참고하여 GAN 논문에 나온 방식과는 약간 다르게 작성하였습니다.
# http://bamos.github.io/2016/08/09/deep-completion/
# 진짜 이미지를 판별하는 D_real 값은 1에 가깝도록,
# 가짜 이미지를 판별하는 D_gene 값은 0에 가깝도록 하는 손실 함수입니다.
loss_D_real = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.zeros_like(D_gene)))
# loss_D_real 과 loss_D_gene 을 더한 뒤 이 값을 최소화 하도록 최적화합니다.
loss_D = loss_D_real + loss_D_gene
# 가짜 이미지를 진짜에 가깝게 만들도록 생성망을 학습시키기 위해, D_gene 을 최대한 1에 가깝도록 만드는 손실함수입니다.
loss_G = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.ones_like(D_gene)))

# TensorFlow 에서 제공하는 유틸리티 함수를 이용해
# discriminator 와 generator scope 에서 사용된 변수들을 쉽게 가져올 수 있습니다.
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                            var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                            var_list=vars_G)

In [8]:
#########
# 신경망 모델 학습
######
generator_saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 편향 초기화
    loss_val_D, loss_val_G = 0, 0

    for epoch in range(total_epoch):
        
        iterator = dataset.make_one_shot_iterator()
        batch_xs, batch_ys = iterator.get_next()
        
        for _ in range(math.ceil(y_len/batch_size)):
            xs, ys = sess.run([batch_xs, batch_ys])
            noise = get_noise(len(ys), n_noise)
            _, loss_val_D = sess.run([train_D, loss_D],
                                     feed_dict={X: xs, Y: ys, Z: noise})
            _, loss_val_G = sess.run([train_G, loss_G],
                                     feed_dict={Y: ys, Z: noise})

        print('Epoch:', '%04d' % epoch,
              'D loss: {:.4}'.format(loss_val_D),
              'G loss: {:.4}'.format(loss_val_G))

    generator_saver.save(sess, './model/')
print('최적화 완료!')

Epoch: 0000 D loss: 2.354e+03 G loss: 0.6321
Epoch: 0001 D loss: 458.7 G loss: 0.483
Epoch: 0002 D loss: 0.9892 G loss: 0.473
Epoch: 0003 D loss: 0.8765 G loss: 0.5488
Epoch: 0004 D loss: 0.7368 G loss: 0.6622
Epoch: 0005 D loss: 0.6049 G loss: 0.7926
Epoch: 0006 D loss: 0.5615 G loss: 0.8511
Epoch: 0007 D loss: 0.481 G loss: 0.9694
Epoch: 0008 D loss: 0.4279 G loss: 1.057
Epoch: 0009 D loss: 0.3354 G loss: 1.268
Epoch: 0010 D loss: 0.2921 G loss: 1.382
Epoch: 0011 D loss: 0.2541 G loss: 1.503
Epoch: 0012 D loss: 0.2429 G loss: 1.551
최적화 완료!


In [10]:
samples = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        sample = sess.run([G],feed_dict={Y: [[0,1]], Z: get_noise(1, n_noise)})
        sample = list(np.reshape(sample,(26)))
        samples.append(sample)
print(np.shape(samples))
   

(100, 26)
